In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0623, -0.2223, -0.0473, -0.2838, -0.0430,  0.2824, -0.0648, -0.1781,
         -0.1555, -0.0877],
        [ 0.0601, -0.1904, -0.0054, -0.3576, -0.1004,  0.2816, -0.1508, -0.1698,
         -0.0573, -0.1281]], grad_fn=<AddmmBackward0>)

In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[ 0.0108, -0.1956, -0.0555, -0.1158,  0.1704,  0.1797, -0.0658, -0.0168,
          0.1524, -0.1903],
        [-0.1287, -0.2541, -0.0659,  0.0889,  0.2514,  0.0180, -0.0806, -0.0016,
         -0.0307, -0.1419]], grad_fn=<AddmmBackward0>)

In [4]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X)

tensor([[ 0.0727, -0.1494,  0.2290, -0.1414,  0.0866, -0.1176,  0.0610, -0.3061,
          0.1342, -0.1805],
        [ 0.0732, -0.1656,  0.2217, -0.0233, -0.0454, -0.0621,  0.0706, -0.2646,
         -0.0679, -0.2397]], grad_fn=<AddmmBackward0>)

In [7]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1698, -0.2842, -0.1841,  0.2658,  0.1992,  0.0830,  0.0945, -0.3605,
         -0.4299,  0.0850],
        [-0.2251, -0.1789, -0.2009,  0.2073,  0.1637,  0.0835,  0.0564, -0.2886,
         -0.2907,  0.0412]], grad_fn=<AddmmBackward0>)

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [10]:
net = FixedHiddenMLP()
net(X)

tensor(0.1820, grad_fn=<SumBackward0>)

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0405, grad_fn=<SumBackward0>)

In [12]:
class ParallelBlock(nn.Module):
    def __init__(self, net1, net2):
        super().__init__()
        self.net1 = net1
        self.net2 = net2
    
    def forward(self, X):
        return self.net2(self.net1(X))
    
net = ParallelBlock(nn.Linear(16, 20), nn.Linear(20, 10))
print(net)
for param in net.parameters():
    print(param)

ParallelBlock(
  (net1): Linear(in_features=16, out_features=20, bias=True)
  (net2): Linear(in_features=20, out_features=10, bias=True)
)
Parameter containing:
tensor([[ 0.0022, -0.2391, -0.1766, -0.1111,  0.1946, -0.1373,  0.1270,  0.1150,
         -0.1431,  0.1772,  0.2208,  0.0518,  0.0910, -0.1587, -0.1977,  0.0252],
        [-0.1535,  0.1074,  0.1740, -0.1748, -0.2230,  0.1172, -0.0341,  0.1435,
          0.1740,  0.0219,  0.1573,  0.0502,  0.1006, -0.1087, -0.0902,  0.1015],
        [-0.0294,  0.1433,  0.0591, -0.0903,  0.1870, -0.0871,  0.0945, -0.1308,
          0.1308, -0.1036, -0.1887, -0.1883, -0.2167, -0.0962,  0.2048,  0.1057],
        [-0.2183,  0.1912,  0.0793, -0.0108, -0.0114,  0.1152, -0.1019, -0.1016,
         -0.0873, -0.0745, -0.0995,  0.0350, -0.2488,  0.0422,  0.1822,  0.0971],
        [-0.1235, -0.1559,  0.0620, -0.1839,  0.2240, -0.2040,  0.1066, -0.1120,
         -0.1873,  0.2370, -0.0728,  0.1199, -0.1927,  0.0527, -0.1473, -0.1268],
        [ 0.0996,  0.049